# PyG vs NeuraLogic TemplateList


Slightly modified https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gcn.py
- Replaced log softmax with sigmoid etc.


In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
import torch_geometric
from torch_geometric.datasets import Planetoid

from neuralogic.utils.templates import TemplateList, GCNConv
from neuralogic.core import Template, Backend, Activation

In [2]:
dataset = 'Cora'

path = osp.join('..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())

data = dataset[0]

In [3]:
def train(model, optimimzer, epochs):
    for epoch in range(1, epochs):
        optimizer.zero_grad()
        F.nll_loss(model(data.x, data.edge_index)[data.train_mask], data.y[data.train_mask]).backward()
        optimizer.step()


@torch.no_grad()
def test(model):
    model.eval()
    logits, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

## PyTorch Geometric

In [4]:
class TorchNet(torch.nn.Module):
    def __init__(self):
        super(TorchNet, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(dataset.num_features, 16, normalize=False, cached=False, bias=False)
        self.conv2 = torch_geometric.nn.GCNConv(16, dataset.num_classes, normalize=False, cached=False, bias=False)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        
        return torch.sigmoid(x)

In [5]:
torch.manual_seed(123)

model = TorchNet()

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters()),
    dict(params=model.conv2.parameters())
], lr=0.01)

train(model, optimizer, 100)

In [6]:
test_results = test(model)

print(test_results)  # train_acc, best_val_acc, test_acc

[0.20714285714285716, 0.108, 0.109]


In [7]:
print(model)

TorchNet(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


## NeuraLogic

In [8]:
torch.manual_seed(123)

template_list = TemplateList([
    GCNConv(in_channels=dataset.num_features, out_channels=16, activation=Activation.RELU),
    GCNConv(in_channels=16, out_channels=dataset.num_classes, activation=Activation.SIGMOID),
])

template = Template(module_list=template_list)
model = template.build(Backend.PYG, native_backend_models=True)

optimizer = torch.optim.Adam([dict(params=model.parameters())], lr=0.01)

train(model, optimizer, 100)

In [9]:
test_results = test(model)

print(test_results)  # train_acc, best_val_acc, test_acc

[0.20714285714285716, 0.108, 0.109]


In [10]:
print(model)

NeuraLogic(
  (module_list): ModuleList(
    (0): GCNConv(1433, 16)
    (1): GCNConv(16, 7)
  )
)
